In [2]:
from google import genai
from google.genai import types
from google.genai.types import GenerateContentConfig
import os

In [24]:
prompt = """
You are a dish idea generator. Your job is to list dishes that can be created with only the ingredients you are given. If a dish cannot be made only with the ingredients given, do not list it! Do not invent new dishes. You do not need to make use of every ingredient. Output as a JSON array. You may assume that the user has basic stuff like water and salt.

Example:
Ingredients:
[
  "Yogurt",
  "Green Apples",
  "Pickles",
  "Donuts",
  "Cheese",
  "Red Apple",
  "Lemon",
  "Cucumber",
  "Lettuce",
  "Eggplant",
  "Bottled Juice",
  "Celery",
  "Limes",
  "Carrots",
  "Oranges",
  "Tomatoes",
  "Green Bell Peppers",
  "Milk"
]

Dishes:
[
  "Yogurt & Fruit Salad",
  "Cucumber & Yogurt Salad",
  "Simple Garden Salad",
  "Pickle & Cheese Plate",
  "Carrot and Celery Sticks with Yogurt Dip",
  "Fruit Smoothie",
  "Lemon-Lime Refresher",
  "Open-Faced Sandwich",
  "Grilled Cheese Sandwich",
  "Cheese and Apple Slices"
]
"""

In [25]:
query = """
Ingredients:
[
  "Yogurt",
  "Green Apples",
  "Pickles",
  "Donuts",
  "Cheese",
  "Red Apple",
  "Lemon",
  "Cucumber",
  "Lettuce",
  "Eggplant",
  "Bottled Juice",
  "Celery",
  "Limes",
  "Carrots",
  "Oranges",
  "Tomatoes",
  "Green Bell Peppers",
  "Milk"
]
"""

In [26]:
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

config = GenerateContentConfig(
    temperature = 0.5,
    max_output_tokens = 2048
)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[prompt, query],
    config=config)

print(response.text)

```json
[
  "Yogurt & Fruit Salad",
  "Cucumber & Yogurt Salad",
  "Simple Garden Salad",
  "Pickle & Cheese Plate",
  "Carrot and Celery Sticks with Yogurt Dip",
  "Fruit Smoothie",
  "Lemon-Lime Refresher",
  "Cheese and Apple Slices",
  "Lettuce and Tomato Salad",
  "Yogurt Drink",
  "Cheese and Pickle Sandwich"
]
```


In [27]:
import json
dishes = json.loads(response.text[response.text.find("["):response.text.rfind("]") + 1])

In [28]:
for dish in dishes:
    print(dish)

Yogurt & Fruit Salad
Cucumber & Yogurt Salad
Simple Garden Salad
Pickle & Cheese Plate
Carrot and Celery Sticks with Yogurt Dip
Fruit Smoothie
Lemon-Lime Refresher
Cheese and Apple Slices
Lettuce and Tomato Salad
Yogurt Drink
Cheese and Pickle Sandwich


In [29]:
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import DuckDuckGoSearchException
import requests
import time
from bs4 import BeautifulSoup

for dish in dishes:
    query = dish
    print(query)
    while True:
        try:
            results = DDGS().text(query, max_results=20)
            break
        except DuckDuckGoSearchException:
            time.sleep(1)

    for result in results[:1]:
        content = requests.get(result['href']).text
        soup = BeautifulSoup(content, 'html.parser')
        content = soup.get_text()
        if "Ingredients" in content:
            ingredient_list_raw = content[content.find("Ingredients"):content.find("Ingredients") + 1000]
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=["What are the ingredients in this recipe?", ingredient_list_raw],
                config=config)
            print(response.text)

Yogurt & Fruit Salad
Cucumber & Yogurt Salad
The ingredients in the recipe are:

*   2 cucumbers
*   1 cup plain yogurt
*   1 teaspoon dried dill (or a couple teaspoons of fresh dill)
*   Salt
*   Pepper
Simple Garden Salad


KeyboardInterrupt: 